In [ ]:
# Загрузка датасета
!pip install -q kagglehub

import kagglehub
import os

path = kagglehub.dataset_download("sanikamal/horses-or-humans-dataset")
train_dir = os.path.join(path, "horse-or-human", "train")
val_dir = os.path.join(path, "horse-or-human", "validation")

Using Colab cache for faster access to the 'horses-or-humans-dataset' dataset.


In [ ]:
# Создание датасетов с аугментацией
import tensorflow as tf

# Аугментация как часть модели
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
])

# Загрузка данных
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.0,
    subset=None,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    validation_split=0.0,
    subset=None,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# Нормализация
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Применяем аугментацию + нормализацию к train
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Оптимизация производительности
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 1027 files belonging to 2 classes.
Found 256 files belonging to 2 classes.


In [ ]:
# Построение CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Обучение
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.5322 - loss: 0.7114 - val_accuracy: 0.5000 - val_loss: 0.7259
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.6147 - loss: 0.6362 - val_accuracy: 0.5000 - val_loss: 0.9556
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.7337 - loss: 0.5408 - val_accuracy: 0.5000 - val_loss: 0.7894
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.7559 - loss: 0.5257 - val_accuracy: 0.5156 - val_loss: 0.7866
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.7148 - loss: 0.5514 - val_accuracy: 0.5156 - val_loss: 0.7422
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.7828 - loss: 0.4471 - val_accuracy: 0.5000 - val_loss: 1.3545
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.8031 - loss: 0.3914 - val_accuracy: 0.5000 - val_loss: 4.3724
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.8285 - loss: 0.3605 - val_accuracy: 0.5000 - val_loss:

In [ ]:
# ROC-AUC
import numpy as np
from sklearn.metrics import roc_auc_score

# Собираем все истинные метки и предсказания
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images, verbose=0).flatten()
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

roc_auc = roc_auc_score(y_true, y_pred)
print(f"\n ROC-AUC: {roc_auc:.4f}")

if roc_auc > 0.6:
    print("Условие выполнено")


 ROC-AUC: 0.8188
Условие выполнено


In [ ]:
# Интерфейс загрузки изображения (работает как раньше)
from google.colab import files
from PIL import Image
import numpy as np

print("Загрузите изображение:")
uploaded = files.upload()

for filename in uploaded.keys():
    img = Image.open(filename).convert("RGB")
    img = img.resize((150, 150))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array, verbose=0)[0][0]
    if pred > 0.5:
        print(f"{filename}: Человек ({pred:.2f})")
    else:
        print(f"{filename}: Лошадь ({1 - pred:.2f})")

Загрузите изображение:


Saving images (4).jpg to images (4) (1).jpg
images (4) (1).jpg: Человек (1.00)
